[View in Colaboratory](https://colab.research.google.com/github/sabumjung/DL-with-TensorFlow/blob/DL-with-TensorFlow/10%EC%9E%A5_2.ipynb)

In [3]:
!pip install gym

In [0]:
import gym
import numpy as np
import tensorflow as tf

In [0]:
# FrozenLake 환경 정의
env = gym.make('FrozenLake-v0')

In [0]:
# 텐서플로 플레이스홀더와 변수 설정
tf.reset_default_graph()
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)

In [0]:
# 손실함수와 최적화방법(GradnentDescentOptimizer) 정의 
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [0]:
# 변수 초기화
init = tf.global_variables_initializer()

In [0]:
# Q-Learning 파라미터 정의
gamma = .99
e = 0.1
num_episodes = 6000
jList = []
rList = []

In [15]:
# Sess실행 및 변수초기화 실시
with tf.Session() as sess:
    sess.run(init)
    # Q-Learning 시작
    for i in range(num_episodes):
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        while j < 99:
            j+=1
            # 여러개의 처리되는 텐서(predict, Qout)를 List로 처리
            # session은 feed일 경우 feed_dict로 처리값을 할당해야 함
            a,allQ = sess.run([predict,Qout],feed_dict= \
                              {inputs1:np.identity(16)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            s1,r,d,_ = env.step(a[0])
            Q1 = sess.run(Qout,feed_dict=\
                          {inputs1:np.identity(16)[s1:s1+1]})
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + gamma *maxQ1
            _,W1 = sess.run([updateModel,W],\
                            feed_dict=\
                           {inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
# 총보상 합산
            rAll += r
            s = s1
            if d == True:
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
# 결과출력
    print("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")            

Percent of succesful episodes: 0.579%
